## DEEP LEARNING Tensor Flow Object Detection

Here's my code for object detection using jupyter notebook. If you're using Windows 10 with Anaconda I recommend use a virtual enviroment and from that, download all the libraries below please refer to https://www.pythond.com/23162/como-activar-un-entorno-anaconda.html for full explanation of how to create a virtual enviroment with conda.

For tensorflow I install the 1.19 because in tensoflow 2.0 some features has changes (i.e detection_graph = tf.Graph() changes by detection_graph = tf.v1.Graph() and ith tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid by  tf.io.gfile.GFile(PATH_TO_CKPT, 'rb') as fid as so on).

In [ ]:
#Basic libraries
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

In [2]:

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image


## Object detection imports


Before run this lines firts install
#pip install tensor-flow-object-detection-api# using Anaconda Prompt in your actual enviroment.

In [3]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [21]:
#Here the webcam is detecting. You can choose VideoCapture(n) with n=0,1. This give the gap variable which port webcam is. In my case I use 0
import cv2
cap = cv2.VideoCapture(0)

# Path to model.
Here you must to add the path to the folder where object_detection is. Make sure that all the information (proto and models) are added to the path


In [ ]:

sys.path.append('C:\\Users\\andre\\Anaconda3\\envs\\opencv-env\\Lib\\site-packages\\tensorf\\models\\research\\object_detection')
    #sys.path.append(LIBRARY_PATH)
    #sys.path.append(LIBRARY_PATH + '/object_detection')

In [23]:

MODEL_NAME = 'ssd_mobilenet_v1_coco_2017_11_17'
MODEL_FILE = MODEL_NAME + '.tar.gz'
DOWNLOAD_BASE = 'http://download.tensorflow.org/models/object_detection/'
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

#Again use PATH to data folder where mscoco_label.pbtxt is, otherwise in label_map will crash
PATH_TO_LABELS = os.path.join('C:\\Users\\andre\\Anaconda3\\envs\\opencv-env\\Lib\\site-packages\\tensorf\\models\\research\\object_detection\\data', 'mscoco_label_map.pbtxt')
NUM_CLASSES = 90
print(PATH_TO_LABELS)

C:\Users\andre\Anaconda3\envs\opencv-env\Lib\site-packages\tensorf\models\research\object_detection\data\mscoco_label_map.pbtxt


In [24]:
opener = urllib.request.URLopener()
opener.retrieve(DOWNLOAD_BASE + MODEL_FILE, MODEL_FILE)
tar_file = tarfile.open(MODEL_FILE)
for file in tar_file.getmembers():
  file_name = os.path.basename(file.name)
  if 'frozen_inference_graph.pb' in file_name:
    tar_file.extract(file, os.getcwd())
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

 ## Loading label map
 Label maps map indices to category names, so that when our convolution network predicts `5`, we know that this corresponds to `airplane`.  Here we use internal utility functions, but anything that returns a dictionary mapping integers to appropriate string labels would be fine

In [25]:

label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

In [26]:
# ## Helper code

# In[8]:

def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)


In [27]:
# # Detection

# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(PATH_TO_TEST_IMAGES_DIR, 'image{}.JPG'.format(i)) for i in range(1, 3) ]

# Size, in inches, of the output images.
IMAGE_SIZE = (12, 8)




In [ ]:
with detection_graph.as_default():
  with tf.Session(graph=detection_graph) as sess:
    while True:
      ret, image_np = cap.read()
      # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
      image_np_expanded = np.expand_dims(image_np, axis=0)
      image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
        
      # Each box represents a part of the image where a particular object was detected.
      boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
        
      # Each score represent how level of confidence for each of the objects.
      # Score is shown on the result image, together with the class label.
      scores = detection_graph.get_tensor_by_name('detection_scores:0')
      classes = detection_graph.get_tensor_by_name('detection_classes:0')
      num_detections = detection_graph.get_tensor_by_name('num_detections:0')
    
    # Actual detection.
      (boxes, scores, classes, num_detections) = sess.run([boxes, scores, classes, num_detections],
                                                          feed_dict={image_tensor: image_np_expanded})
      # Visualization of the results of a detection.
      vis_util.visualize_boxes_and_labels_on_image_array(image_np,np.squeeze(boxes),np.squeeze(classes).astype(np.int32),
          np.squeeze(scores),
          category_index,
          use_normalized_coordinates=True,
          line_thickness=8)

      cv2.imshow('object detection', cv2.resize(image_np, (800,600)))
      if cv2.waitKey(25) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        break
      